# InfCtx trainer baseline setup
The trainer validation is mostly done using the same settings as raven 1B5 model.
- Layer count: 24
- Embed size: 2048

Typically with the following dataset
- "teven/enwiki_10k" dataset, chunked to 1024 token sizes

The following notebook, helps perform the basic download and setup for the "init model", and "test dataset". Which is used as a reference point for all other validation processes (unless stated otherwise)

Generally you only need to do this once

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps
>
> All training runs (except dryrun) is configured to log to weights and bias, comment out the logger in the config file if you want to avoid this

## Preparing the init model and test dataset

In [1]:
# First lets setup the various directories, and get the blank init model, these init model was generated
# using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# As such I have preinitialized these blank models and uploaded them to HF for convinence
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/
!rm -rf ../../model/Echo-A-1B5-Init.pth
!cd ../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
!ls -alh ../../model/Echo-A-1B5-Init.pth

--2023-07-09 05:50:16--  https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Init.pth
Resolving huggingface.co (huggingface.co)... 65.8.243.16, 65.8.243.46, 65.8.243.92, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/cb/ef/cbef09abb2634a3375b28868bffa285226dfeabedec89b28c2fb302221164d66/0ec7214ed16737a6348254e6f96d8cdc04d3b5efbd5f53fe9337607ea42b5b9f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Echo-A-1B5-Init.pth%3B+filename%3D%22Echo-A-1B5-Init.pth%22%3B&Expires=1689141016&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTE0MTAxNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jYi9lZi9jYmVmMDlhYmIyNjM0YTMzNzViMjg4NjhiZmZhMjg1MjI2ZGZlYWJlZGVjODliMjhjMmZiMzAyMjIxMTY0ZDY2LzBlYzcyMTRlZDE2NzM3YTYzNDgyNTRlNmY5NmQ4Y2RjMDRkM2I1ZWZiZDVmNTNmZT

In [2]:
# Lets preload the requried dataset
!cd ../../RWKV-v4neo && python3 preload_dataset.py ../notebook/trainer-validation/config/baseline-dryrun.yaml

Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 1218.92it/s]
Setting num_proc from 26 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Dataset parquet downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 330.75it/s]


# Trainer Code validation via dryrun

The following dryrun, help do a basic check that the existing trainer code changes are valid across 2 * 2 data samples.

If this check fail, its most probably a code / envrionment setup issue (no further checks needed)

It does not log the run the W&B

In [1]:
# Validate source code and env is working, by doing a short 2 sample dryrun
!cd ../../RWKV-v4neo && python3 new_train.py fit -c ../notebook/trainer-validation/config/baseline-dryrun.yaml

[2023-07-09 06:12:04,496] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model via the following optimization mode 'torch-compile' with torch '2.1.0.dev20230708'
Global seed set to 3941088705
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_128_bf16/build.ninja...
Building extension module wkv_128_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_128_bf16...
^C
Traceback (most recent call last):
  File "/home/ubuntu/dev-infctx-torch-compile/RWKV-v4neo/new_train.py", line 15, in <module>
    cli_main()
  File "/home/ubuntu/dev-infctx-torch-compile/RWKV-v4neo/new_train.py", line 8, in cli_main
    LightningCLI(
  File "/home/ubuntu/anaconda3/env

# (Optional) Baseline full context (1024) training

Perform a full 1 epoch training run of training context size = 1024. Ensuring all data samples fit within the allocated training size. And is used as the baseline loss comparision for several experiments

In [9]:
# Full training run
!cd ../../RWKV-v4neo && python3 new_train.py fit -c ../notebook/trainer-validation/config/baseline-1024.yaml

[2023-07-01 20:53:18,310] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230701_205320-k8flu72z
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-validation-full (train-ctx=1024, data-ctx=1024, bs=12)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/k8flu72z
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_1024_bf16/build.ninja...
Building extension module wkv_1024_bf16...
Allowing ninja to set a default number of workers... (overridable by